# Clasificacion de la calidad del vino

Se entrenara un modelo que clasifique la calidad del vino, ya sea tinto o blanco, segun sus caracteristicas

In [1]:
# librerias
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split as tts

## Datos

In [2]:
tinto=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv',
                  sep=';')

print (tinto.shape)
tinto.head()

(1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
blanco=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
                  sep=';')

print (blanco.shape)
blanco.head()

In [ ]:
# se concatena en un dataframe

blanco['color']=0
tinto['color']=1

vino=pd.concat([blanco, tinto], ignore_index=True)
vino.head()

**exploracion**

In [ ]:
vino.columns

In [ ]:
vino.describe()

In [ ]:
vino.info(memory_usage='deep')

In [ ]:
# dato objetivo (calidad)

vino.quality.value_counts()

In [ ]:
vino.quality.hist()
plt.title('Histograma Calidad del Vino')
plt.xlabel('Grupos de Calidad')
plt.ylabel('Numero de elementos')
plt.show();

Los datos estan desbalanceados, tanto en tinto-blanco como en las calidades.

Se hace un nuevo binning, llamando calidad=3 a lo que eran (3,4,5). calidad=6, (6). calidad=9, (7,8,9). De esta manera quedan balanceados segun calidad. Es una simple aproximacion a este problema, la idea es llegar a un modelo funcional para poner en produccion. 


In [ ]:
# ahora solo tres clases
vino_malo=vino[vino.quality.isin([3,4,5])]
vino_malo.quality=3

vino_normal=vino[vino.quality==6]
vino_normal.quality=6

vino_bueno= vino[vino.quality.isin([7,8,9])]
vino_bueno.quality=9

vino=pd.concat([vino_malo,
                vino_normal,
                vino_bueno], 
               ignore_index=True)

# se guarda
vino.to_csv('data/vino_data.csv', index=False)

vino.quality.value_counts()

In [ ]:
vino.quality.hist()
plt.title('Histograma Calidad del Vino')
plt.xlabel('Grupos de Calidad')
plt.ylabel('Numero de elementos')
plt.show();

In [ ]:
vino.head()

## Modelo

In [ ]:
# train-test split

X=vino.drop('quality', axis=1)

vino.quality=pd.Categorical(vino.quality)
vino['quality_c']=vino.quality.cat.codes
y=vino.quality_c

In [ ]:
X_train, X_test, y_train, y_test=tts(X, y, test_size=0.2, random_state=42)

print (X_train.shape, X_test.shape)

In [ ]:
# clasificador

gbc=GBC(random_state=10, learning_rate=0.1, max_depth=10)

gbc.fit(X_train, y_train)

# se guarda el entrenamiento en pickle para no entrenar en la web
with open('data/gbc.p', 'wb') as f:
    pickle.dump(gbc, f, 2)

# se carga el modelo
gbc=pickle.load(open('data/gbc.p', 'rb' ))

In [ ]:
y_pred=gbc.predict(X_test)
y_pred

In [ ]:
y_prob=gbc.predict_proba(X_test)
y_prob

**Evaluacion**

In [ ]:
# accuracy

gbc.score(X_test, y_test)

**Matriz de Confusion**

In [ ]:
# plot matriz confusion

confusion=confusion_matrix(y_test, y_pred)

ax=sns.heatmap(confusion,
            cmap=plt.cm.Blues,
            annot=True)

b, t=ax.get_ylim()
ax.set_ylim(b+.5, t-.5)

plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.title('Confusion Matrix(clases)')
plt.show()

In [ ]:
ax=sns.heatmap((confusion/len(y_pred)*100),
                cmap=plt.cm.Blues,
                annot=True)

b, t=ax.get_ylim()
ax.set_ylim(b+.5, t-.5)

plt.xlabel('Prediction')
plt.ylabel('Truth')
plt.title('Confusion Matrix(%)')
plt.show()

## Prediciendo a mano

In [ ]:
# nuevos datos, la media

n_data={k:v for k,v in zip(X_train.columns, vino.mean())}
n_data['color']=0
n_data

In [ ]:
# se pasa a dataframe

X_pred=pd.DataFrame.from_dict(n_data, orient='index').T
X_pred

In [ ]:
# prediccion clases=[0,1,2]

pred=gbc.predict_proba(X_pred)
pred

# Web-App 

## main.py

In [ ]:
# librerias
from flask import Flask
from flask import render_template
from flask import flash
from flask import request
from flask import jsonify
from flask import Markup

import logging
import io
import os
import sys

import pandas as pd
import numpy as np
import scipy
import pickle



# inicia flask
app=Flask(__name__)



# modelo 
gbc=None


# variables, caracteristicas
var=None
 

# ruta absoluta
#PATH=os.path.dirname(os.path.abspath(__file__))  # para servidor
PATH=''   

# para cargar las imagenes
def imagen_vino(color, calidad):
    if color==0:
        color_str='blanco'
    else:
        color_str='tinto'
    return('/static/images/vino_' +color_str+'_'+str(calidad)+'.jpg')




# antes del primer request
@app.before_first_request
def startup():
    global gbc
    gbc=pickle.load(open(PATH+'data/gbc.p','rb'))
    global var
    var=pd.read_csv(PATH+'data/vino_data.csv').drop('quality', axis=1).columns



# manejo de errores
@app.errorhandler(500)
def server_error(e):
    logging.exception('algun error...')
    return """
    And internal error <pre>{}</pre>
    """.format(e), 500



# conexion a traves de ruta
@app.route('/backend', methods=['POST', 'GET'])
def backend():
    # requests
    req=[request.args.get(e.replace(' ', '_')) for e in var]
    
    # nuevos datos
    n_data={k:v for k,v in zip(var, req)}
    n_data['color']=int(request.args.get('color'))
    
    X_pred=pd.DataFrame.from_dict(n_data, orient='index').T

    # prediccion
    prob=gbc.predict_proba(X_pred)

    pred=[3,6,9][np.argmax(prob[0])]
    
    return jsonify({'prediccion':pred, 
                    'imagen': imagen_vino(n_data['color'], pred)})


# principal
@app.route("/", methods=['POST', 'GET'])
def main():
    logging.warning('index!')
    # carga por defecto
    return render_template('index.html', 
                           prediccion=1, 
                           imagen=PATH+'/static/images/vino_tinto_6.jpg')





# solo en local
if __name__=='__main__':
    app.run(debug=False)


## Codigo HTML